In [1]:
# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
# 导入本次需要的 package 和 标准库
import json   # json 标准库
import pandas as pd
import numpy  as np
from bs4 import BeautifulSoup
from xml.etree import cElementTree as ET  # XML 标准库

# 正则表达式的标准库
import re

# mapbox 是 mapbox 提供的 Python API
from mapbox import Datasets
from mapbox import Geocoder

## API 获取数据

在示例[Google 地图 API 演示](http://www.runoob.com/try/try.php?filename=tryhtml_map_first)中是通过 API 请求数据来渲染地图信息。此外 API 同样是可以获取以及提交数据信息的，下面将使用 [Mapbox](https://www.mapbox.com/api-documentation/?language=Python#introduction) API 来提取数据。其中 Datasets 信息可以通过 [Datasets | Mapbox](https://www.mapbox.com/studio/datasets/)查询比对：
![image-20190111142204688](https://ws1.sinaimg.cn/large/006tNc79gy1fz2mib3myfj30lz05bmxc.jpg)


通过调用 Geocoder 的类方法，可以查询国家地区的数据，例如 `forward()` 方法

In [ ]:
# 准备信息，例如 token，可能还需要账户名
if False:
    access_token = input("Enter your API token:\t")
else:
    with open("./test_token_key.txt", "r") as file:
        access_token = file.read().strip()

In [ ]:
# 查询个人保存的 dataset 信息
datasets = Datasets(access_token=access_token)
datasets.list().json()

In [ ]:
geocoder = Geocoder(access_token=access_token)

# 使用 forward 方法来获取需要的信息
province = geocoder.forward("sichuan", country=["cn"])
province.geojson()

## XML 与 HTML
### XML 文件
不论是 XML 还是 HTML 首先需要理解文件结构：

```{xml}
<?xml version="1.0"?>
<data>
  <country name="Liechtenstein">
    <rank>1</rank>
    <year>2008</year>
    <gdppc>141100</gdppc>
    <neighbor name="Austria" direction="E"/>
    <neighbor name="Switzerland" direction="W"/>
  </country>
  <country name="Singapore">
    <rank>4</rank>
    <year>2011</year>
    <gdppc>59900</gdppc>
    <neighbor name="Malaysia" direction="N"/>
  </country>
  <country name="Panama">
    <rank>68</rank>
    <year>2011</year>
    <gdppc>13600</gdppc>
    <neighbor name="Costa Rica" direction="W"/>
    <neighbor name="Colombia" direction="E"/>
  </country>
</data>

```
需要在理解文件的结构的基础上，去解析数据内容。获取数据的过程中，需要注意 `attribute-value` 和 `text` 两个属性。因此对于 xml 文件读取过程中，注意使用 `getroot()`, `getchildren()` 等方法来访问文件相应的标签，以及使用 `attrib`， `text`来获取相应的数值——前者得到的是一个字典样式的数据，后者是一个数据值

In [ ]:
# 通过 parse 方法来解析文件
tree = ET.parse("./data/country_data.xml")

In [ ]:
# 通过 getroot 获取数据的 root，这样得到完整的数据
root = tree.getroot()

print("Get the root:\n", root)

In [ ]:
for child in root.getchildren():
    # country 的属性 name 是对应的值，没有其他 text 值
    print("Attribute country:\t", child.attrib)
    for low_child in child.findall("year"):
        # 在每个 country 下，year 有相应的 text 值，而无其他属性值
        print("Country year:\t", low_child.text)

### HTML 文件
HTML 文件在结构上和 XML 相似，所以这里不再讲解关于结构上的问题，而主要关注 HTML 中的三个重要的三个，Element、Tag 以及 Attribute，关于三者的内容可以从下图中去了解：
![](https://www.techfry.com/images/php/html-element.gif)
其中 Attribute 部分需要注意使用 Class 和 ID 进行元素筛选时——ID 具有唯一性，使用 `find_all()`, `find()` 等方法——需要注意传入参数；在进行节点选择时，使用 `find_parents()`, `find_parent()`, `find_next_sibling()` , `next_siblings`, `previous_element` 等方法或者属性进行遍历或者搜索，详情参考 [Beautiful Soup 文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html)

In [ ]:
from pprint import pprint

In [ ]:
with open("./data/beautifule.html", "r") as file:
    soup = BeautifulSoup(file, "lxml")

In [ ]:
internal_link = soup.find_all("a", class_="reference internal")
internal_link

In [ ]:
internal_link[0].text

In [ ]:
# 获取属性值
internal_link[0].attrs

In [ ]:
# 使用 ID 值筛选
encoding_id = soup.find_all(id="encodings")
len(encoding_id)

In [ ]:
encoding_id[0].contents

## Json 文件
### Json 基本结构
首先来了解 json 文件的结构，json 有存储形式存在差异，下面是以 array 和 字典形式（即 **records** 形式存储）——` [{column -> value}, ... , {column -> value}]`
```{json}
[
    {
        "Team Name": "Leicester City",
        "Wins": 23,
        "Draws": 12,
        "Losses": 3
    },
    {
        "Team Name": "Arsenal",
        "Wins": 20,
        "Draws": 11,
        "Losses": 7
    }
]
```
此外还有以字典嵌套形式（即 **index** ——`{index -> {column -> value}}` 或者 **columns** ——`{column -> {index -> value}}` 形式存储）
```{json}
{
    "Draws": {
        "0": 11,
        "1": 13,
    },
    "Losses": {
        "0": 7,
        "1": 6,
    },
    "Team Name": {
        "0": "Arsenal",
        "1": "Tottenham Hotspur",
    },
    "Wins": {
        "0": 20,
        "1": 19,
    }
}
```

In [ ]:
filename = "./data/league.json"

In [ ]:
df = pd.read_json(filename, orient="record")

In [ ]:
df.head()

### Json 嵌套数据
前面已经探讨过了，对于 json 数据的基本结构来说，它是一个字典类型为基础的数据格式——这点是从 Python 的角度来说。对于复杂的嵌套数据，需要从外向内访问以得到需要的数据值，遵循基本的访问数据方式：

* 字典，通常通过键访问数据
* 列表和元组，通常通过索引下标的方式访问数据

对于嵌套数据的实际例子中，<font color="cornflowerblue">需要额外注意<b>异常处理</b>——例如字典中可能存在某些键不在所有的数据中，这是最常见的问题</font>

```
{"web-app": {
  "servlet": [   
    {
      "servlet-name": "cofaxCDS",
      "servlet-class": "org.cofax.cds.CDSServlet",
      "init-param": {
        "configGlossary:installationAt": "Philadelphia, PA",
        "configGlossary:adminEmail": "ksm@pobox.com",
        "configGlossary:poweredBy": "Cofax",
        "maxUrlLength": 500}},
    {
      "servlet-name": "cofaxEmail",
      "servlet-class": "org.cofax.cds.EmailServlet",
      "init-param": {
      "mailHost": "mail1",
      "mailHostOverride": "mail2"}},
    {
      "servlet-name": "cofaxAdmin",
      "servlet-class": "org.cofax.cds.AdminServlet"},
 
    {
      "servlet-name": "fileServlet",
      "servlet-class": "org.cofax.cds.FileServlet"}],
}
    
```

In [ ]:
with open("./data/app_data.json", "r") as file:
    data = json.load(file)

In [ ]:
data["web-app"]["servlet"]

In [ ]:
# 异常处理，需要使用 try-except 语句。某些建不是所有的数据都是拥有的
raw_data = dict(
    servlet_name=[],
    dataStoreMaxConns=[], 
    dataStoreLogFile=[],
    poweredByIcon=[],
    servlet_class=[]
    )
key_value = {
    "dataStoreMaxConns": "dataStoreMaxConns", 
    "dataStoreLogFile": "dataStoreLogFile", 
    "configGlossary:poweredByIcon": "poweredByIcon",
}

In [ ]:
for item in data["web-app"]["servlet"]:
    raw_data["servlet_name"].append(item["servlet-name"])
    raw_data["servlet_class"].append(item["servlet-class"])
    for key in key_value:
        try:
            raw_data[key_value[key]].append(item["init-param"][key])
        except KeyError:
            raw_data[key_value[key]].append(np.nan)

In [ ]:
raw_data

In [ ]:
for key in key_value:
    try:
        for item in data["web-app"]["servlet"]:
            raw_data[key_value[key]].append()
    except KeyError:
        raw_data[key_value[key]].append(np.nan)

In [ ]:
raw_data

In [ ]:
raw_data.keys()

此外对于单独每一行存储了 JSON 对象数据的文件，可以直接通过调整 `read_json()` 的 `lines` 参数来读取数据。例如下面的 `lines_json.txt` 文件的存储内容形式为：

```
{"Draws":12,"Losses":3,"Team Name":"Leicester City","Wins":23}
{"Draws":11,"Losses":7,"Team Name":"Arsenal","Wins":20}
{"Draws":13,"Losses":6,"Team Name":"Tottenham Hotspur","Wins":19}
{"Draws":9,"Losses":10,"Team Name":"Manchester City","Wins":19}
{"Draws":9,"Losses":10,"Team Name":"Manchester United","Wins":19}
{"Draws":9,"Losses":11,"Team Name":"Southampton","Wins":18}
{"Draws":14,"Losses":8,"Team Name":"West Ham United","Wins":16}
```
可以通过调整 `lines=True` 的参数——默认值是 `False`，来读取数据这样就可以直接读取到 `txt` 中的数据内容

In [ ]:
try:
    print(pd.read_json("./data/lines_json.txt", lines=False).head())
except ValueError:
    print("Default method can't parse the file correctly")

In [ ]:
pd.read_json("./data/lines_json.txt", lines=True).head()

### Json 库处理文件
对于数据结构整齐的数据，可以直接通过 pandas 的 read_json 方法来获取。对于数据结构不整齐或者想要获得指定的数据，那么可以参考访问字典的方式现将数据筛选出来之后再进行相关的处理。下面将进行指定数据提取的方式获取数据

In [ ]:
with open(filename, "r") as file:
    print(type(file))
    data = json.load(file)

In [ ]:
with open(filename, "r") as file:
    raw_data = file.read()
    print(type(raw_data), raw_data)
    data = json.loads(raw_data)

In [ ]:
data

In [ ]:
data_selected = dict()

for i in data:
    

## 正则表达式

通过一定的规则来描述或匹配字符串。所以在使用正则表达式过程中需要了解创建模式的基本符号——  `\w \W \d \D \s \S \t \r \n {} [] * + .`

### Word
* `\w` 用于匹配字符、数字以及 _
* `\W` 用于匹配不符合 `\w` 的字符

In [ ]:
text = "\\1word_ s\tt\nn"

def single_reg_search(pattern, text=text):
    pattern = re.compile(pattern)
    group = pattern.search(text)
    
    print(group.group())

In [ ]:
# \w 的匹配
text = "\\1word_ s\tt\nn"

single_reg_search(r"\w", text)

In [ ]:
# \W 的匹配
text = "\\1word_ s\tt\nn"
single_reg_search(r"\W", text)

### Digit
* `\d` 用于匹配数字，包括 0 到 9
* `\D` 用于匹配非数字

In [ ]:
# \d 的匹配
text = "\\1word_ s\tt\nn"
single_reg_search(r"\d", text)

In [ ]:
# \D 的匹配
text = "\\1word_ s\tt\nn"
single_reg_search(r"\D", text)

### Space And New Line

* ` ` 匹配空格
* `\t` 匹配一个 Tab 键位
* `\n` 匹配换行符
* `\s` 匹配空格、`\t`、`\n`
* `\S` 匹配非 `\s`

In [ ]:
# 匹配 \t
text = "\\1word_ s\tt\nn"
single_reg_search(r"\t\w", text)

In [ ]:
# 匹配 \n
text = "\\1word_ s\tt\nn"
single_reg_search(r"\n\w", text)

In [ ]:
# 匹配 \s
text = "\\1word_ s\tt\nn"
single_reg_search(r"\s\w", text)

In [ ]:
# 匹配 \S
text = "\\1word_ s\tt\nn"
single_reg_search(r"\S")

### 其他标识符

* `[]` 表示匹配的范围
* `{}` 表示匹配的数量
* `()` 构造一个组
* `？` 表示匹配零次和一次
* `+` 表示匹配一次或者多次
* `*` 表示零次或者多次
* `|` 表示 `|` 前后两种情况均可以使用
* `^` 有两种功能，一种是表示从行首；另一种是结合其他标识符，表示否定 `[^aeiou]`表示不筛选元音字符
* `$` 表示行尾

In [ ]:
# 匹配元音字母
text = "\\1word_ s\tt\nn"

pattern = re.compile(r'[aeiou]')
pattern.search(text).group()

In [ ]:
# 匹配多个字符

text = "\\1word_ s\tt\nn"

# 匹配一个
pattern = re.compile(r'\w{1}')
pattern.search(text).group()

In [ ]:
# 匹配至少一个
pattern = re.compile(r'\w{1,}')
pattern.search(text).group()

In [ ]:
# 匹配一个或者三个
pattern = re.compile(r'\w{1,3}')
pattern.search(text).group()

In [ ]:
# 使用 + 匹配一次或者多次——解决示例，提取数据值
text = r"word 121\23.4 \t\s"
pattern = re.compile(r"\d+.*\d")

pattern.search(text).group()

In [ ]:
pattern.groups

### `re` 标准库方法
在使用 re 库时，可以通过 `search()`, `match()`, `findall()` 进行搜索匹配；使用 `group(n)` 方法可以返回指定匹配字符。此外需要注意一下，`findall()` 方法返回的结果是一个列表。更多信息详见：

#### 匹配方法
`match()` 和 `search()` 区别：前者开头与正则表达式不匹配，直接返回None，而后者则是全字符串“扫描”

* `re.match(pattern, string, flags=0)`

    参数：匹配的正则表达式，要匹配的字符串，标志位(匹配方法) 尝试从字符串的开头进行匹配，匹配成功会返回一个匹配的对象，类型是：<class '_sre.SRE_Match'>。可以使用 `group()` 与 `groups()` 方法

* `re.search(pattern, string, flags=0)`

    参数：同上,扫描整个字符串，可以使用 `group()` 与 `groups()` 方法

#### 检索
```
re.findall(pattern, string, flags=0)

    参数：同上
    遍历字符串，找到正则表达式匹配的所有位置，并以列表的形式返回

re.finditer(pattern, string, flags=0)

    参数：同上
    遍历字符串，找到正则表达式匹配的所有位置，并以迭代器的形式返回
```

#### 替换
```
re.sub(pattern, repl, string, count=0, flags=0)

    参数：repl替换为什么字符串，可以是函数，把匹配到的结果做一些转换；
    count替换的最大次数，默认0代表替换所有的匹配。
    找到所有匹配的子字符串，并替换为新的内容
re.subn(pattern, repl, string, count=0, flags=0)
    功能同上面的 sub 相同，只是返回的是一个元组
```    
#### 拆分
```
re.split(pattern, string, maxsplit=0, flags=0)

    参数：maxsplit设置分割的数量，默认0代表所有满足匹配的都分割
    在正则表达式匹配的地方进行分割，并返回一个列表
```

#### 常用匹配结果查询方法
* `group()` 方法 用于确认满足匹配模式的结果，可以传入多个匹配结果的索引值——`match.group(1, 2)`。`group()` 可以得到没有进行分组的匹配结果—— _换句话说，就是可以尝试使用 `group()` 方法来确认 pattern 是否正确_ 。 只有所有的匹配模式都无法匹配时，结果为 `None`
* `groups()` 方法 用于返回包含所有匹配分组的元组—— _注意所有的结果是需要在分组中，所以满足某些模式的字符串但是该方法可能不会有结果_ 。 此外对于分组结果中是 `None` 的结果，可以使用传入其他值来替换——`m.groups("a")`，这样的情况下如果结果是 `None` 的分组会被 `a` 替换。<font size=1.5 color="red">警示信息</font><font color="white">；此外多个解释的可能行匹配模式，得到的结果中不一定是所有的分组都参与其中</font>。
* `groupdict()` 方法 返回一个关于被命名的匹配对象的字典，键是匹配模式的命名字符串，值是匹配的字符串

In [ ]:
# match 是表示从头开始搜索
text = r"word 121\23.4 \t\s"
pattern = re.compile(r"\d+.*\d")

print(pattern.match(text))

In [ ]:
# search 表示完整搜索
text = r"word 121\23.4 \t\s"
pattern = re.compile(r"\d+.*\d")

pattern.search(text).group()

In [ ]:
# 使用 findall 方法搜索
text = r"word 121\23.4 \t\s"
pattern = re.compile(r"\d+.*\d")

pattern.findall(text)

### 组合方法使用
正则表达式的强大功能在于将以上标识符和其他字符组合使用，这样构造出强大的模式以此来进行筛选或者替换等工作。此外需要注意，在使用 `compile()` 方法时，可以传入 `re.IGNORECASE` 来表示忽略大小写

#### 常规组合方法

In [ ]:
text = r"wordWORD 121\23.4 \t\s"

pattern = re.compile(r"(word)+", re.IGNORECASE)

pattern.search(text).group()

In [ ]:
# 使用 findall 方法搜索，仅提取出数字
text = r"word 121\23.4 \t\s"
pattern = re.compile(r"(\d+\.?\d+)")

pattern.findall(text)

#### `()` 使用
为了筛选出复合特定组合的字符串，使用 `()` 来构建一个子表达式。这个子表达式可以结合其他标识符构建出更强大功能的模式，例如结合表达次数的标识符，可以表示需要满足匹配次数。此外还有一方面是某些方法需要构建分组才能运行，例如 Series 中使用 str 下的 extract 方法。

* 结合次数标识符表示相同模式出现次数
* 对模式进行命名，可以方便筛选特定的组——`(?P<name>pattern)`。在 Series 中使用该方法可以将模式名称赋值给列名称
* 结合 `?` 和 `:` 构建匹配的模式——`(?:pattern)`，但是不捕获该模式——主要反应在匹配组的数量或者说索引上

<font color="red" size=1.5>警示信息</font>

<font color="white">结合 <code style="color:#EFF0F1">?</code> 和 <code style="color:#EFF0F1">=</code> 构建匹配模式——<code style="color:#EFF0F1">(?=pattern)</code>，作用是匹配模式之前的字符串，但是条件是有这个 pattern 
* 结合 <code style="color:#EFF0F1">?</code> , <code style="color:#EFF0F1"><</code> 和 <code style="color:#EFF0F1">=</code> 构建匹配模式——<code style="color:#EFF0F1">(?<=pattern)</code>，作用是匹配模式之后的字符串，但条件是有这个 pattern</font>

In [ ]:
# 结合次数表示分组重复次数——解决 IP 地址筛选
text = "010.179.000.001__010.183.255.254__gansu____"
pattern = re.compile(r"(\d+(\.\d+){3})")

pattern.search(text).group()

In [ ]:
# 使用 (?P<name>pattern) 的模式，进行匹配和捕获到特定的 name 下
pattern = re.compile(r"(?P<ip>\d+(\.\d+){3})")

for i in pattern.finditer(text):
    print("Groups result:\n\t", i.groups())
    print("Group result:\n\t", i.group())
    print("Group result with key\n\t", i.group("ip"))
    print("Group dict result:\n\t", i.groupdict())

In [ ]:
# 使用 (?:pattern) 的模式，匹配但不捕获
text = "010.179.000.001__010.183.255.254__gansu____210"
pattern = re.compile(r"(?:\d+(?:\.\d+){3})(\w+)$")
pattern.search(text).groups()

In [ ]:
result = pattern.search(text)
for index in range(result.lastindex+1):
    print("当前的匹配组索引值 {0}， 值为 {1}".format(index, result.group(index)))

In [ ]:
# 使用 (?:pattern) 的模式，这里是尝试使用更多的组
pattern = re.compile(r"(?:\d+(\.\d+){3})(\w+)$")
pattern.search(text).groups()

In [ ]:
result = pattern.search(text)
for index in range(result.lastindex+1):
    print("当前的匹配组索引值 {0}， 值为 {1}".format(index, result.group(index)))

In [ ]:
# 使用 (?=pattern) 的模式，匹配 pattern 之前的字符串
# 注意匹配模式上只是得到中间部分的 IP
pattern = re.compile(r"[0-9\.]+(?=[a-z_]+\d+$)")
pattern.search(text).group()

In [ ]:
# 使用 (?=pattern) 的模式，匹配 pattern 之前的字符串
# 注意匹配模式上只是针对中间部分的 IP
# ⚠️这个只是了解
text = "010.179.000.001__010.183.255.254__gansu____210"
pattern = re.compile(r"([0-9\.]+)__(?=[0-9\.]+)(\w+\.\w+)")
pattern.search(text).groups()

In [ ]:
# 使用 (?<=pattern) 的模式，匹配 pattern 之后的字符串
# 可以匹配模式之后的字符串
# ⚠️这个只是做了解
text = "ThisIsTest010.183.255.254++gansu++"
pattern = re.compile(r"(\w+)(?<=[0-9\.]{4})([+a-z]+)")
pattern.search(text).groups()

In [ ]:
# 使用 (?<=pattern) 的模式，匹配 pattern 之前的字符串
# 和上面的结果相同
text = "ThisIsTest010.183.255.254++gansu++"
pattern = re.compile(r"(?<=[0-9\.]{4})(\w+)([+a-z]+)")
pattern.search(text).groups()

### Series 中的正则表达式使用
Series 的字符串数据可以通过调用 `.str` 的属性来使用处理字符串相关的方法，例如：

* `.str.strip()` 方法可以删除字符串两端空格, `\t` , `\n`
* `.str.cat()` 方法可以用于拼接包含字符串列表。可以拼接有两种方式，一种是自拼接——这样可以将整列字符串合并；另一种和其他具有同样长度的 Series，Index，DattaFrame 以及其他列表数据拼接


In [ ]:
df = pd.read_csv("./data/string_series.csv")

In [ ]:
df.head(2)

In [ ]:
# 自拼接，sep 表示分隔方式
df.offer_type.str.cat(sep=", ")

In [ ]:
# 和其他数据进行拼接
df.offer_type.str.cat(df, sep=", ")

除了以上的字符串处理，在处理 Series 中的字符串同样可以直接使用正则表达式来进行搜索、替换以及提取相关信息：

* `str.extract()` 提取正则表达式中匹配分组的模式
* `str.extractall()` 提取所有满足分组模式字符串
* `str.findall()` 搜索满足模式的字符串或者正则表达式。注意和 `str.find()` 存在差异，该方法得到的结果是搜索到的自字符串的索引值

In [ ]:
# 加载数据内容
with open("./data/movies.dat", "r", encoding="ISO-8859-1") as file:
    data = [i.strip() for i in file.readlines()]

In [ ]:
series = pd.Series(data)

In [ ]:
series.head()

In [ ]:
# 使用 extract 提取满足分组中的第一个模式的字符串——如果下面使用 ([a-z\s]+)|\((\d+)\)
# 结果只能得到标题，如果没有标题时才会返回第二年份
series.str.extract(r"([a-z\s]+)\((\d+)\)", flags=re.IGNORECASE).head()

In [ ]:
# 提前赋予列名称
series.str.extract(r"(?P<name>[a-z\s]+)\((?P<year>\d+)\)", flags=re.IGNORECASE).head()

In [ ]:
series.str.extractall(r"([a-z\s]+|\d+\))", flags=re.I).head(10)

In [ ]:
# findall 提取所有满足模式的所有字符串
series.str.findall(r"(?P<name>[a-z\s]+)\((?P<year>\d+)\)", flags=re.I).head(10)

#### 多条件模式—— `|` 
使用 `A|B` 可以将在同一个分组或者单一正则表达式下使用多个条件，该方式表示的是匹配两者中的一种方案即可。

In [ ]:
series.str.findall(r"[a-z\s]+|\d{4}", flags=re.I).head(10)

## 疑问
### 非英文字符匹配
解决这个问题的核心是参考 Unicode 的字符集编码范围，因为中文字符的编码范围在 `[\u4E00-\u9FEF]`，所以在进行中文匹配时可以使用 `"[\u4E00-\u9FEF]"` 的模式来匹配中文文字

In [ ]:
re.search(r"[\u4E00-\u9FEF]*","你好est")

## 参考
1. [JSON Example](https://json.org/example.html)
2. [正则表达式30分钟入门教程](http://deerchao.net/tutorials/regex/regex.htm)
3. [学习正则表达式的简单方法 learn-regex 中文版本](https://juejin.im/entry/5993b56f6fb9a0248f4a5f16#3-%E7%AE%80%E5%86%99%E5%AD%97%E7%AC%A6%E9%9B%86)
4. [正则表达式学习笔记](http://notes.maxwi.com/2015/10/04/Regular-Expression-Study-Note/)
5. [正则表达式练习题](http://notes.maxwi.com/2015/10/06/Regex-practice/) 收集了一些文档数据
6. [Code Charts](http://www.unicode.org/charts/) Unicode 字符集编码表——11.0 Version
7. [CJK Unified Ideographs (Han)](http://www.unicode.org/charts/PDF/U4E00.pdf) Unicode 中文编码集
8. [Regular Expression: Match Unicode Block Range](http://kourge.net/projects/regexp-unicode-block) 可以搜索其他字符集。注意⚠️中文字符集名称是 CJK Unified Ideographs
9. [Pythex: a Python regular expression editor](https://pythex.org/) 可以在线测试正则表达式的结果，可以显示捕获结果和匹配结果
10. [Tutorials | Mapbox](https://www.mapbox.com/help/tutorials/)
11. [Mapbox API Documentation](https://www.mapbox.com/api-documentation/?language=Python#introduction)